In [0]:
create streaming table customers as
select * from cloud_files("abfss://raw@adlsdatamaster.dfs.core.windows.net/dlt/customers", "csv")

In [0]:
-- Create and populate the target table.
CREATE OR REFRESH STREAMING TABLE customer_silver;

APPLY CHANGES INTO
  customer_silver
FROM
  stream(live.customers)
KEYS
  (customer_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
SEQUENCE BY
  sequenceNum
COLUMNS * EXCEPT
  (operation, sequenceNum,_rescued_data)
STORED AS
  SCD TYPE 2;

In [0]:
create streaming table customer_silver_active as 
select customer_id, customer_name, customer_email, customer_city, customer_state from stream(live.customer_silver) where __END_AT Is null

In [0]:
create streaming table sales as
select * from cloud_files("abfss://raw@adlsdatamaster.dfs.core.windows.net/dlt/sales", "csv")

In [0]:
create streaming table sales_sivler
(
  CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION DROP ROW
) 
as 
select distinct * from stream(live.sales )

In [0]:
create streaming table products as
select * from cloud_files("abfss://raw@adlsdatamaster.dfs.core.windows.net/dlt/product", "csv")

In [0]:
-- Create and populate the target table.
CREATE OR REFRESH STREAMING TABLE products_silver;

APPLY CHANGES INTO
  products_silver
FROM
  stream(live.products)
KEYS
  (product_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
SEQUENCE BY
  seqNum
COLUMNS * EXCEPT
  (operation, seqNum,_rescued_data)
STORED AS
  SCD TYPE 1;

In [0]:
create streaming table customers_total_amount as 
select s.customer_id, c.customer_name, s.total_amount
from stream(live.sales_sivler )as s 
left join live.customer_silver_active as c
on s.customer_id = c.customer_id